In [4]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [17]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg_path = 'yolof_r50_c5_8x8_1x_coco.py'
cfg = Config.fromfile('./' + cfg_path)

root='../dataset/'

# # dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'train.json' # valid json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

# cfg.data.samples_per_gpu = 4

cfg.seed = 2021
cfg.gpu_ids = [0]
cfg.work_dir = '../work_dirs/' + cfg_path[:-3]

cfg.model.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.model.keys()

dict_keys(['type', 'backbone', 'neck', 'bbox_head', 'train_cfg', 'test_cfg'])

In [18]:
cfg.keys()

dict_keys(['dataset_type', 'data_root', 'img_norm_cfg', 'train_pipeline', 'test_pipeline', 'data', 'evaluation', 'optimizer', 'optimizer_config', 'lr_config', 'runner', 'checkpoint_config', 'log_config', 'custom_hooks', 'dist_params', 'log_level', 'load_from', 'resume_from', 'workflow', 'model', 'seed', 'gpu_ids', 'work_dir'])

In [12]:
cfg.data.test

{'type': 'CocoDataset',
 'ann_file': '../dataset/train.json',
 'img_prefix': '../dataset/',
 'pipeline': [{'type': 'LoadImageFromFile'},
  {'type': 'MultiScaleFlipAug',
   'img_scale': (512, 512),
   'flip': False,
   'transforms': [{'type': 'Resize', 'keep_ratio': True},
    {'type': 'RandomFlip'},
    {'type': 'Normalize',
     'mean': [103.53, 116.28, 123.675],
     'std': [1.0, 1.0, 1.0],
     'to_rgb': False},
    {'type': 'Pad', 'size_divisor': 32},
    {'type': 'ImageToTensor', 'keys': ['img']},
    {'type': 'Collect', 'keys': ['img']}]}],
 'classes': ('General trash',
  'Paper',
  'Paper pack',
  'Metal',
  'Glass',
  'Plastic',
  'Styrofoam',
  'Plastic bag',
  'Battery',
  'Clothing')}

In [19]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.19s)
creating index...
index created!


In [20]:
# dataset 확인
datasets

[
 CocoDataset Train dataset with number of images 4883, and instance counts: 
 +-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
 | category          | count | category      | count | category        | count | category    | count | category     | count |
 +-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
 | 0 [General trash] | 3965  | 1 [Paper]     | 6352  | 2 [Paper pack]  | 897   | 3 [Metal]   | 936   | 4 [Glass]    | 982   |
 | 5 [Plastic]       | 2943  | 6 [Styrofoam] | 1263  | 7 [Plastic bag] | 5178  | 8 [Battery] | 159   | 9 [Clothing] | 468   |
 +-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+]

In [21]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

/opt/conda/envs/detection/lib/python3.7/site-packages/mmdet/core/anchor/builder.py:17: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  '``build_anchor_generator`` would be deprecated soon, please use '
2021-10-10 08:45:59,590 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'open-mmlab://detectron/resnet50_caffe'}
2021-10-10 08:45:59,592 - mmcv - INFO - load model from: open-mmlab://detectron/resnet50_caffe
2021-10-10 08:45:59,592 - mmcv - INFO - Use load_from_openmmlab loader
2021-10-10 08:45:59,701 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: conv1.bias, layer1.0.conv1.bias, layer1.0.conv2.bias, layer1.0.conv3.bias, layer1.0.downsample.0.bias, layer1.1.conv1.bias, layer1.1.conv2.bias, layer1.1.conv3.bias, layer1.2.conv1.bias, layer1.2.conv2.bias, layer1.2.conv3.bias, layer2.0.conv1.bias, layer2.0.conv2.bias, layer2.0.conv3.bias

In [22]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=False)


2021-10-10 08:46:05,001 - mmdet - INFO - Start running, host: root@ae472b7aee24, work_dir: /opt/ml/detection/work_dirs/my_config_1010
2021-10-10 08:46:05,002 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_NORMAL) OptimizerHook                      
(NORMAL      ) CheckpointHook                     
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHoo